In [24]:
from langchain.chains import LLMChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key='chat_history',
    return_messages=True,
)
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are a helpful AI talking to a human. '),
        MessagesPlaceholder(variable_name='chat_history'),
        ('human', '{question}'),
    ]
)
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)


chain.invoke({'question': "I'm Flynn."})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human. 
Human: I'm Flynn.

> Finished chain.


{'question': "I'm Flynn.",
 'chat_history': [HumanMessage(content="I'm Flynn."),
  AIMessage(content='Hello Flynn! How can I assist you today?')],
 'text': 'Hello Flynn! How can I assist you today?'}

In [25]:
chain.invoke({'question': "I live in Seoul."})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human. 
Human: I'm Flynn.
AI: Hello Flynn! How can I assist you today?
Human: I live in Seoul.

> Finished chain.


{'question': 'I live in Seoul.',
 'chat_history': [HumanMessage(content="I'm Flynn."),
  AIMessage(content='Hello Flynn! How can I assist you today?'),
  HumanMessage(content='I live in Seoul.'),
  AIMessage(content="That's great to know, Seoul is a vibrant city with a rich culture and history. Is there anything specific you'd like to know or talk about regarding Seoul?")],
 'text': "That's great to know, Seoul is a vibrant city with a rich culture and history. Is there anything specific you'd like to know or talk about regarding Seoul?"}

In [26]:
chain.invoke({'question': "I'm a software engineer."})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human. 
Human: I'm Flynn.
AI: Hello Flynn! How can I assist you today?
Human: I live in Seoul.
AI: That's great to know, Seoul is a vibrant city with a rich culture and history. Is there anything specific you'd like to know or talk about regarding Seoul?
Human: I'm a software engineer.

> Finished chain.


{'question': "I'm a software engineer.",
 'chat_history': [HumanMessage(content="I'm Flynn."),
  AIMessage(content='Hello Flynn! How can I assist you today?'),
  HumanMessage(content='I live in Seoul.'),
  AIMessage(content="That's great to know, Seoul is a vibrant city with a rich culture and history. Is there anything specific you'd like to know or talk about regarding Seoul?"),
  HumanMessage(content="I'm a software engineer."),
  AIMessage(content="That's fantastic, Flynn! As a software engineer, you must work on interesting projects. Is there a particular aspect of software engineering that you find most fascinating or challenging?")],
 'text': "That's fantastic, Flynn! As a software engineer, you must work on interesting projects. Is there a particular aspect of software engineering that you find most fascinating or challenging?"}

In [27]:
chain.invoke({'question': "Who am I?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human. 
Human: I'm Flynn.
AI: Hello Flynn! How can I assist you today?
Human: I live in Seoul.
AI: That's great to know, Seoul is a vibrant city with a rich culture and history. Is there anything specific you'd like to know or talk about regarding Seoul?
Human: I'm a software engineer.
AI: That's fantastic, Flynn! As a software engineer, you must work on interesting projects. Is there a particular aspect of software engineering that you find most fascinating or challenging?
Human: Who am I?

> Finished chain.


{'question': 'Who am I?',
 'chat_history': [HumanMessage(content="I'm a software engineer."),
  AIMessage(content="That's fantastic, Flynn! As a software engineer, you must work on interesting projects. Is there a particular aspect of software engineering that you find most fascinating or challenging?"),
  HumanMessage(content='Who am I?'),
  AIMessage(content="You are Flynn, a software engineer living in Seoul. You seem to have a passion for technology and software development. Is there anything else you'd like to share or discuss?")],
 'text': "You are Flynn, a software engineer living in Seoul. You seem to have a passion for technology and software development. Is there anything else you'd like to share or discuss?"}